# [IAPR][iapr]: Project


**Group ID:** 26

**Author 1 (sciper):** Andrea Oliveri (283506)  
**Author 2 (sciper):** Charles Berger (272018)   
**Author 3 (sciper):** Chun Hei Michael Chan (311722)  

**Release date:** 07.05.2021  
**Due date:** 03.06.2021 (23h59)


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class as well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

---
## 0. Introduction

An anonymous researcher that we will name Lann Yecun is convinced that the MNIST dataset still has great potential. He decides to create a playing card game based on MNIST digits and different figures. The game uses a standard 52 card deck which is composed of four French suits/colours: clubs (&#9827;), diamonds (&#9830;), hearts (&#9829;) and spades (&#9824;). Each suit includes 10 digit cards (from 0 to 9) and 3 figures (Jack-J, Queen-Q, and King-K). Here is an example of the 13 spade cards with their name.


<img src="media/example_cards.png">


We can find the same arrangement of cards for the clubs, diamonds, and hearts. 


## 1. Rules


### 1.1 Standard

The rules are based on the simple battle card game. The goal of the game is to win as many points as possible. Each turn, the 4 players play a card in front of them. As displayed in the example below. The rules are the following:

- The cards are ranked in the following order : **0 < 1 < 2 < 3 < 4 < 5 < 6 < 7 < 8 < 9 < J < Q < K**.
- The player with the highest-ranked card wins the round and obtains **1 point**. 
- If the highest-ranked card is the same for multiple players we call it a draw and all winners get **1 points**. 
- In this configuration, we **do not** take into account the suits. The game only rely on the card ranks. 
- The game lasts 13 rounds. After the last round, the winner is the player that has the largest number of points. 
- In the example below Player 1 wins the round with his Queen ( 0 < 8 < J < **Q**).

If two or more players have the same number of points they share the victory.

### 1.2 Advanced

The advanced rules take into account the suits. 

- At the beginning of **each round** a random player is designated as the **dealer**. The dealer places a green token with the letter *D* next to him (player 1 in the example below).
- Only the cards that belong to the same suit as the one of the dealer are considered valid. In the example below, only Player 4 is competing with Player 1 as spade was selected by the dealer (e.i., Player 1). Player 2 and 3 are out for this round. Player 1 wins the round and **1 point** with the Queen ( 0&#9824; < **Q&#9824;**).
- There cannot be any draw between the players as they are not any card duplicates.
- We use the same system as the standard method to count the points.


<img src="media/example_round.jpg">


### 1.3 Notes

- The orientation of the card is linked to the position of the player around the table. For instance, to read the card of the 3rd player you will have to rotate it by 180°.
- The **digits** always **face** the players around the table. The figures can have random orientations.
- Player 1 **always** seats south of the table. The players are **always** ordered counter-clockwise as in the example. 
- The dealers can change between the rounds and games.
- Some cards **might** apear multiple times per game.
- Pictures are always taken from rougthly the same altitude.
- The digits from the training set **would not** be the same as the one of the testing set.

---
## 2. Data

You will be given the images of 7 games that were played ([download link](https://drive.google.com/drive/folders/1fEy27wnJsUJPRsEEomzoAtP56s-7HFtk?usp=sharing)). The data are composed of:
   - 7 folder named after the games (game1 to game7).
   - Each game includes 13 ordered images (1st to 13th round).
   - Each game includes a csv file with the ground truth of the game. The first row list the players (P1 to P4) as well as the dealer (D). The following rows represent the rounds (1 to 13). We represent the card played with 2 character as $AB$ where $A \in [0-9, J, Q, K]$ is the rank of the card and $B \in [C, D, H, S]$ is the suit. For example, QS means "(Q)ueen of (S)pade" and 0D means "(0) of (D)iamond". The dealer is represented by the ID of the player (e.g. P1 -> 1).
   
You are free to use external datasets such as the original MNIST train set that you used in lab 3.

---
## 3. Your Tasks

Your task is to ready yourself for the final evaluation. The day of the exam we will give you a new folder with a new game. ! The digits on the cards **differ** from the one of the traning set. When given a new data folder with 13 images your should be able to:

**Task 0**
   - Plot an overlay for each round image that shows your detections and classification. You can for example plot bounding boxes around the cards/dealer token and add a text overlay with the name of the classes.

**Task 1**
   - (a) Predict the **rank** of the card played by each player at each round (Standard rules).
   - (b) Predict the **number of points** of each player according to **Standard** rules
 
**Task 2**
   - (a) Detect which player is the selected **dealer** for each round.
   - (b) Predict the **rank** and the **suit** of the card played by each player at each round (Advanced rules).
   - (c) Predict the **number of points** of each player according to **Advanced** rules

---

**Before the exam (until 03.06.21 at 23h59)**
   - Create a zipped folder named **group_xx.zip** that you uplaod on moodel (xx being your group number).
   - Include a **runnable** code (Jupyter Notebook and external files) and your presentation in the zip folder.
   
**The day of the exam (04.06.21)**
   - You will be given a **new folder** with 13 images (rounds) and but **no ground truth** (csv file).
   - We will ask you to run your pipeline in **realtime** and to send us your prediction of task 1 and 2 that you obtain with the function **print_results**. 
   - On our side we will compute the perfomance of your classification algorithm. 
   - To evaluate your method we will use the **evaluate_game** function presented below. To understant how the provided functions work please read the documentation of the functions in **utils.py**.
   - **Please make sure your function returns the proper data format to avoid points penalty the day of the exam**. 

In [46]:
from scripts import *
import tensorflow as tf
import pickle


# FUNCTIONS BELOW MIGHT NEED TO BE SHIFTED TO UTILITIES.PY OR SOMEWHERE ELSE
def process_figure_mask(mask):
    height, width = mask.shape
    max_size = max(height, width)
    
    pad_left   = (max_size - width ) // 2
    pad_right  = max_size - width - pad_left
    pad_top    = (max_size - height) // 2
    pad_bottom = max_size - height - pad_top
    
    mask = cv2.copyMakeBorder(mask, pad_top, pad_bottom, pad_left, pad_right, borderType = cv2.BORDER_CONSTANT, value = 0)
    mask = zoom_image_to_meet_shape(mask, (28, 28))
    mask = normalize(mask)
    mask = binarize(mask)
    
    return mask.astype(np.float32)

suit_translate = {0: 'S', 1: 'C', 2: 'H', 3: 'D'}
figure_translate = {i:str(i) for i in range(10)}
figure_translate[10] = 'J'
figure_translate[11] = 'Q'
figure_translate[12] = 'K'


def round2df(dealer, class_fig, class_suit, rd_nb, game_nb):
    # dealer -> number, class_fig -> array[4], class_suit -> array[4] 
    # rd_nb -> number, game_nb -> number
    
    suits = [suit_translate[c] for c in class_suit]
    figures = [figure_translate[c] for c in class_fig]
    round_label = {'P'+str(i+1):[figures[i]+suits[i]] for i in range(len(suits))}
    round_label['D'] = [dealer]
    round_label['round'] = rd_nb
    round_label['game'] = game_nb
    
    return pd.DataFrame.from_dict(round_label)

def pred_to_answer(fig_class, suit_class, dealer):
    # format of answer is :
    # two lists of -> rank (e.g '0H') and -> dealer (e.g 2)
    pass

<b> 1. We load the data i.e the images and the ground truth. We as well instantiate the extractor. </b>

In [47]:
# use loader to get labels and images

data_loader = DataLoader()
extractor = Extractor()

<b> 2. We load the trained models for prediction </b>

In [48]:
# Loading models

figure_clf = tf.keras.models.load_model('./Models/Fully Convolutional Network')
with open('model_suit.pickle', 'rb') as handle:
    clf = pickle.load(handle)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fcf9f963c40> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fcf9fa82ca0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fcf9fa829a0> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fcf9fa7e4c0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fcf9fa7e040> and <tensorflow.python.keras.layers.core.SpatialDropout2D object at 0x7fcf9fa920a0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fcf9fa92160> and <tensorflow.python.keras.layers.core.Flatten object at 0x7fcf9fafd250>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object a

<b> 3. We obtain the predictions from the extracted images, extracted by extractor</b>


In [51]:
from tqdm import tqdm
# Obtain predictions for all images

res = []
for game in tqdm(data_loader.get_available_games()):
    round_fig_preds, round_suit_preds = [], []
    round_dealer = []
    for round_ in data_loader.get_available_rounds(game):
        # print("Game: {}, Round: {}".format(game, round_))
        
        image, _ = data_loader[game, round_]
        # we use here the extractor object
        dealer, cards, figures_suits = extractor(image)
        
        class_fig  = []
        class_suit = []
        
        fig_suits = sorted(list(figures_suits.items()),key=lambda x:x[0])
        for player, images in fig_suits:
            figure_mask = get_color_pixels(images['figure'], images['color'])
            suits_mask  = get_color_pixels(images['suit'  ], images['color'])
            figure_mask = process_figure_mask(figure_mask)
        
            pred = figure_clf(figure_mask[None, :, :, None])
            pred = np.argmax(pred)
            pred_suits = suit_predict(images['suit'], clf)
            
            if verbose:
                plt.imshow(figure_mask)
                plt.title(pred)
                plt.show()
                
            class_fig.append(pred)
            class_suit.append(pred_suits)
            

        class_suit = np.asarray(class_suit).squeeze()
        df_round = round2df(dealer, class_fig, class_suit, round_, game)
        res.append(df_round)

df_res = pd.concat(res)

100%|██████████| 7/7 [01:46<00:00, 15.21s/it]


<b>4. Verify the different accuracies achieved on the 6 games by our method </b>

Get accuracies according to rules

In [68]:
for GAME_IDX in range(1,7):
    # Load ground truth from game 1
    cgt = pd.read_csv('train_games/game{0}/game{0}.csv'.format(GAME_IDX), index_col=0)
    cgt_rank = cgt[['P1', 'P2', 'P3', 'P4']].values
    pred_rank = np.asarray(df_res[df_res['game']==GAME_IDX][['P1','P2','P3','P4']])

    # Compute accuracy of prediction
    acc_standard = evaluate_game(pred_rank, cgt_rank, mode_advanced=False)
    acc_advanced = evaluate_game(pred_rank, cgt_rank, mode_advanced=True)
    print("Your model accuracy for game {} is: Standard={:.3f}, Advanced={:.3f}".format(GAME_IDX,
                                                                            acc_standard, 
                                                                            acc_advanced))

Your model accuracy for game 1 is: Standard=1.000, Advanced=1.000
Your model accuracy for game 2 is: Standard=1.000, Advanced=1.000
Your model accuracy for game 3 is: Standard=1.000, Advanced=1.000
Your model accuracy for game 4 is: Standard=1.000, Advanced=1.000
Your model accuracy for game 5 is: Standard=1.000, Advanced=1.000
Your model accuracy for game 6 is: Standard=1.000, Advanced=1.000


Print the results on one game

In [64]:
GAME_IDX = 7 

g = Game(df_res.loc[df_res['game']==GAME_IDX])

# Creates dummy predictions (toy exmaple)
pred_rank = np.asarray(df_res[df_res['game']==1][['P1','P2','P3','P4']])
pred_dealer = list(df_res[df_res['game']==1]['D'])

pred_pts_stand = g.gamewinner(rule='standard')
pred_pts_advan = g.gamewinner(rule='advanced')

print('count points (stadard rule): {}'.format(g.gamewinner(rule='standard')))
print('count points (advanced rule): {}'.format(g.gamewinner(rule='advanced')))

count points (stadard rule): [0, 4, 4, 5]
count points (advanced rule): [4, 2, 4, 3]


In [65]:
print_results(
    rank_colour=pred_rank, 
    dealer=pred_dealer, 
    pts_standard=pred_pts_stand,
    pts_advanced=pred_pts_advan,
)

The cards played were:
[
['QS', '8H', 'JH', '0S'], 
['5S', 'JS', '9C', 'KD'], 
['1D', '3H', '7C', '3D'], 
['1S', 'JD', '4S', '4C'], 
['0D', '6S', '3C', '2C'], 
['8S', '9D', '3S', 'KS'], 
['4D', '7H', '7D', '6H'], 
['0C', 'KC', '9S', '8C'], 
['5D', '5H', '1H', '7S'], 
['QH', 'QC', '6D', '9H'], 
['4H', '2D', 'JC', 'QD'], 
['6C', '8D', '2H', 'KH'], 
['1C', '5C', '0H', '2S'], 
]
Players designated as dealer: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Players points (standard): [0, 4, 4, 5]
Players points (advanced): [4, 2, 4, 3]


---
### 3.1 Example Final results

Example of output you **should** provide the day of the final exam.

In [1]:
from utils import print_results
import numpy as np

# Creates dummy predictions (toy exmaple)
pred_rank = np.array(["0D"]*4*13).reshape((13, 4)) # Everyone played the "0 of spade".
pred_dealer = [1]*13                # List of players selected as dealer for each round
pred_pts_stand = [0,0,0,13]         # Player 4 won 13 points with standard rules.
pred_pts_advan = [0,0,8,7]          # Player 3 and 4 won 8 and 7 points with adv, rules respectively.

print_results(
    rank_colour=pred_rank, 
    dealer=pred_dealer, 
    pts_standard=pred_pts_stand,
    pts_advanced=pred_pts_advan,
)

The cards played were:
[
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
['0D', '0D', '0D', '0D'], 
]
Players designated as dealer: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Players points (standard): [0, 0, 0, 13]
Players points (advanced): [0, 0, 8, 7]


---
### 3.2 Example Accuracy

Example of code you can use to validate the performance of your model. Be careful the day of the exam you will not have access to the ground truth of the predictions.

In [2]:
from utils import evaluate_game
import pandas as pd
#! You will need pandas libary to run the example. Please install the package using pip or conda commands !

# Load ground truth from game 1
cgt = pd.read_csv('train_games/game1/game1.csv', index_col=0)
cgt_rank = cgt[['P1', 'P2', 'P3', 'P4']].values

# Compute accuracy of prediction
acc_standard = evaluate_game(pred_rank, cgt_rank, mode_advanced=False)
acc_advanced = evaluate_game(pred_rank, cgt_rank, mode_advanced=True)
print("Your model accuracy is: Standard={:.3f}, Advanced={:.3f}".format(acc_standard, acc_advanced))

Your model accuracy is: Standard=0.077, Advanced=0.019
